In [37]:
# Install dependencies
!pip install fastf1 tabulate

# Import the essentials
import os
import fastf1
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from tabulate import tabulate

In [38]:
# Cache setup—keeps fastf1 from choking
cache_dir = "/content/f1_cache"
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)
fastf1.Cache.enable_cache(cache_dir)

# 2025 driver grid—hardcoded lineup
grid_2025 = pd.DataFrame({
    "Driver": ["Lando Norris", "Oscar Piastri", "Max Verstappen", "Liam Lawson",
               "Lewis Hamilton", "Charles Leclerc", "Alex Albon", "Carlos Sainz",
               "George Russell", "Kimi Antonelli", "Fernando Alonso", "Lance Stroll",
               "Pierre Gasly", "Jack Doohan", "Yuki Tsunoda", "Isack Hadjar",
               "Oliver Bearman", "Esteban Ocon", "Nico Hulkenberg", "Gabriel Bortoleto"],
    "DriverCode": ["NOR", "PIA", "VER", "LAW", "HAM", "LEC", "ALB", "SAI", "RUS", "ANT",
                   "ALO", "STR", "GAS", "DOO", "TSU", "HAD", "BEA", "OCO", "HUL", "BOR"],
    "DriverNumber": ['4', '81', '1', '30', '44', '16', '23', '55', '63', '87',
                     '14', '18', '10', '12', '22', '6', '7', '31', '27', '5'],
    "TeamStrength": [9.8, 9.8, 9.2, 9.2, 7.5, 7.5, 7.8, 7.8, 8.8, 8.8,
                     6.8, 6.8, 6.0, 6.0, 5.5, 5.5, 5.0, 5.0, 4.5, 4.5],  # Arbitrary team ratings
    "DriverExperience": [6, 2, 10, 1, 18, 7, 6, 10, 6, 1, 22, 7, 8, 1, 5, 1, 1, 8, 12, 1]  # Years on the grid
})

# 2024 average positions—past performance baseline
season_2024_avg = {
    '1': 1.5, '4': 3.0, '81': 5.0, '44': 4.0, '16': 6.0, '55': 7.0, '63': 2.5, '23': 8.0,
    '14': 10.0, '18': 12.0, '10': 13.0, '22': 11.0, '31': 9.0, '27': 14.0,
    '30': 15.0, '87': 16.0, '12': 17.0, '6': 18.0, '7': 19.0, '5': 20.0
}
grid_2025["Avg2024Position"] = grid_2025["DriverNumber"].map(season_2024_avg)  # Link to drivers

In [39]:
# Blind prediction for Aus 2025—no real data, just vibes
features = ["Avg2024Position", "TeamStrength", "DriverExperience"]
X_blind = grid_2025[features].fillna(grid_2025[features].median())  # Handle missing values
model = RandomForestRegressor(n_estimators=100, random_state=42)  # 100-tree forest
model.fit(X_blind, X_blind["Avg2024Position"])  # Train on 2024 data
grid_2025["PredictedAusPositionFloat"] = model.predict(X_blind)  # Raw prediction scores
grid_2025["PredictedAusPosition"] = grid_2025["PredictedAusPositionFloat"].rank(method="first").astype(int)  # Convert to ranks
grid_2025 = grid_2025.sort_values("PredictedAusPosition")  # Order by position

# Output Aus blind prediction—clean and sharp
print("\n🎉 Blind Prediction for 2025 Australian GP 🎉\n")
print(tabulate(grid_2025[["Driver", "DriverNumber", "PredictedAusPosition"]].head(10),
               headers=["Driver", "No.", "Pred. Pos."], tablefmt="fancy_grid", showindex=False))

# Pull real Aus 2025 race data from fastf1
session_aus_2025 = fastf1.get_session(2025, 'Australian', 'R')  # Race session
session_aus_2025.load()  # Fetch it
aus_2025_results = session_aus_2025.results[["DriverNumber", "Position"]].rename(columns={"Position": "ActualPosition"})  # Extract positions
aus_2025_results["ActualPosition"] = pd.to_numeric(aus_2025_results["ActualPosition"], errors='coerce')  # Numeric conversion
aus_2025_results["Weather"] = 1  # Wet race flag
grid_2025 = grid_2025.merge(aus_2025_results, on="DriverNumber", how="left")  # Combine with grid

# Compare blind prediction to reality
print("\n🏁 Comparison with Actual Aus 2025 Results 🏁\n")
print(tabulate(grid_2025[["Driver", "DriverNumber", "PredictedAusPosition", "ActualPosition"]].head(10),
               headers=["Driver", "No.", "Pred. Pos.", "Act. Pos."], tablefmt="pretty", showindex=False))

# Calculate error—how far off we were
mae = mean_absolute_error(grid_2025["ActualPosition"].fillna(20), grid_2025["PredictedAusPosition"])  # DNFs as 20
print(f"\n🌟 Mean Absolute Error for Aus Prediction: {mae:.2f} 🌟")


core           INFO 	Loading data for Australian Grand Prix - Race [v3.5.3]
INFO:fastf1.fastf1.core:Loading data for Australian Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
INFO:fastf1.fastf1.req:Using cached data for session_info
req            INFO 	Using cached data for driver_info
INFO:fastf1.fastf1.req:Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
INFO:fastf1.fastf1.req:Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
INFO:fastf1.fastf1.req:Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
INFO:fastf1.fastf1.req:Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
INFO:fastf1.fastf1.req:Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
INFO:fastf1.fastf1.req:Using cached data for timing_app_data
core   


🎉 Blind Prediction for 2025 Australian GP 🎉

╒═════════════════╤═══════╤══════════════╕
│ Driver          │   No. │   Pred. Pos. │
╞═════════════════╪═══════╪══════════════╡
│ Max Verstappen  │     1 │            1 │
├─────────────────┼───────┼──────────────┤
│ George Russell  │    63 │            2 │
├─────────────────┼───────┼──────────────┤
│ Lando Norris    │     4 │            3 │
├─────────────────┼───────┼──────────────┤
│ Lewis Hamilton  │    44 │            4 │
├─────────────────┼───────┼──────────────┤
│ Oscar Piastri   │    81 │            5 │
├─────────────────┼───────┼──────────────┤
│ Charles Leclerc │    16 │            6 │
├─────────────────┼───────┼──────────────┤
│ Carlos Sainz    │    55 │            7 │
├─────────────────┼───────┼──────────────┤
│ Alex Albon      │    23 │            8 │
├─────────────────┼───────┼──────────────┤
│ Esteban Ocon    │    31 │            9 │
├─────────────────┼───────┼──────────────┤
│ Fernando Alonso │    14 │           10 │
╘═══════

req            INFO 	Using cached data for car_data
INFO:fastf1.fastf1.req:Using cached data for car_data
req            INFO 	Using cached data for position_data
INFO:fastf1.fastf1.req:Using cached data for position_data
req            INFO 	Using cached data for weather_data
INFO:fastf1.fastf1.req:Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
INFO:fastf1.fastf1.req:Using cached data for race_control_messages
core        WARNING 	Driver 4 completed the race distance 00:00.022000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['4', '1', '63', '12', '23', '18', '27', '16', '81', '44', '10', '22', '31', '87', '30', '5', '14', '55', '7', '6']
INFO:fastf1.fastf1.core:Finished loading data for 20 drivers: ['4', '1', '63', '12', '23', '18', '27', '16', '81', '44', '10', '22', '31', '87', '30', '5', '14', '55', '7', '6']



🏁 Comparison with Actual Aus 2025 Results 🏁

+-----------------+-----+------------+-----------+
|     Driver      | No. | Pred. Pos. | Act. Pos. |
+-----------------+-----+------------+-----------+
| Max Verstappen  |  1  |     1      |    2.0    |
| George Russell  | 63  |     2      |    3.0    |
|  Lando Norris   |  4  |     3      |    1.0    |
| Lewis Hamilton  | 44  |     4      |   10.0    |
|  Oscar Piastri  | 81  |     5      |    9.0    |
| Charles Leclerc | 16  |     6      |    8.0    |
|  Carlos Sainz   | 55  |     7      |   18.0    |
|   Alex Albon    | 23  |     8      |    5.0    |
|  Esteban Ocon   | 31  |     9      |   13.0    |
| Fernando Alonso | 14  |     10     |   17.0    |
+-----------------+-----+------------+-----------+

🌟 Mean Absolute Error for Aus Prediction: 3.90 🌟


In [40]:
# Load China 2025 Sprint Qualifying data
session_2025_sprint_quali = fastf1.get_session(2025, 'Chinese', 'Sprint Qualifying')  # Sprint session
session_2025_sprint_quali.load()  # Get the data
sprint_quali_2025 = session_2025_sprint_quali.laps[["DriverNumber", "LapTime"]].groupby("DriverNumber").agg({"LapTime": "min"}).reset_index()  # Best laps
sprint_quali_2025["SprintQualiTime (s)"] = sprint_quali_2025["LapTime"].dt.total_seconds()  # Seconds format

# Set up China race prediction data
data_2025 = grid_2025.drop(columns=["PredictedAusPosition", "PredictedAusPositionFloat", "ActualPosition"])  # Reset
data_2025 = data_2025.merge(sprint_quali_2025[["DriverNumber", "SprintQualiTime (s)"]], on="DriverNumber", how="left")  # Add Sprint
data_2025["SprintQualiDelta"] = (data_2025["SprintQualiTime (s)"] - data_2025["SprintQualiTime (s)"].min()) / data_2025["SprintQualiTime (s)"].min()  # Pace gap
data_2025["Weather"] = 0  # Dry race assumption
data_2025["SprintRank"] = data_2025["SprintQualiTime (s)"].rank(method="first").astype(int)  # Sprint order

# Train model for China race—blend Aus and Sprint
features = ["Avg2024Position", "TeamStrength", "DriverExperience", "Weather", "SprintQualiDelta"]
X_train = data_2025[features].fillna(data_2025[features].median())  # Prep training data
y_train = 0.2 * grid_2025["ActualPosition"].fillna(20) + 0.8 * data_2025["SprintRank"]  # 20% Aus, 80% Sprint
model.fit(X_train, y_train)  # Fit the model
print("\n✨ Feature Importances for China ✨")
print(tabulate(pd.DataFrame(list(dict(zip(features, model.feature_importances_)).items()), columns=["Feature", "Importance"]),
               headers="keys", tablefmt="psql", showindex=False))  # Feature weights

core           INFO 	Loading data for Chinese Grand Prix - Sprint Qualifying [v3.5.3]
INFO:fastf1.fastf1.core:Loading data for Chinese Grand Prix - Sprint Qualifying [v3.5.3]
req            INFO 	Using cached data for session_info
INFO:fastf1.fastf1.req:Using cached data for session_info
req            INFO 	Using cached data for driver_info
INFO:fastf1.fastf1.req:Using cached data for driver_info
core        WARNING 	Sprint Qualifying is not supported by Ergast! Limited results are calculated from timing data.
req            INFO 	Using cached data for session_status_data
INFO:fastf1.fastf1.req:Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
INFO:fastf1.fastf1.req:Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
INFO:fastf1.fastf1.req:Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
INFO:fastf1.fastf1.req:Using cached data


✨ Feature Importances for China ✨
+------------------+--------------+
| Feature          |   Importance |
|------------------+--------------|
| Avg2024Position  |    0.0785697 |
| TeamStrength     |    0.0348889 |
| DriverExperience |    0.0129494 |
| Weather          |    0         |
| SprintQualiDelta |    0.873592  |
+------------------+--------------+


In [41]:
# Predict China race grid
X_pred = data_2025[features].fillna(data_2025[features].median())  # Prediction data
data_2025["PredictedPositionFloat"] = model.predict(X_pred)  # Raw scores
data_2025["PredictedPosition"] = data_2025["PredictedPositionFloat"].rank(method="first").astype(int)  # Final ranks
data_2025 = data_2025.sort_values("PredictedPosition")  # Sort it

# Display the China race prediction—sleek and bold
print("\n🏆 Predicted 2025 Chinese GP Full Grid 🏆\n")
print(tabulate(data_2025[["Driver", "DriverNumber", "SprintQualiTime (s)", "SprintRank", "PredictedPosition"]].head(20),
               headers=["Driver", "No.", "Sprint Time (s)", "Sprint Rank", "Pred. Pos."], tablefmt="fancy_grid", showindex=False,
               floatfmt=".3f"))

# Save to CSV—keep the proof
data_2025.to_csv("f1_2025_season_data.csv", index=False)


🏆 Predicted 2025 Chinese GP Full Grid 🏆

╒═══════════════════╤═══════╤═══════════════════╤═══════════════╤══════════════╕
│ Driver            │   No. │   Sprint Time (s) │   Sprint Rank │   Pred. Pos. │
╞═══════════════════╪═══════╪═══════════════════╪═══════════════╪══════════════╡
│ Max Verstappen    │     1 │            90.867 │             2 │            1 │
├───────────────────┼───────┼───────────────────┼───────────────┼──────────────┤
│ Lewis Hamilton    │    44 │            90.849 │             1 │            2 │
├───────────────────┼───────┼───────────────────┼───────────────┼──────────────┤
│ Oscar Piastri     │    81 │            90.929 │             3 │            3 │
├───────────────────┼───────┼───────────────────┼───────────────┼──────────────┤
│ George Russell    │    63 │            91.169 │             5 │            4 │
├───────────────────┼───────┼───────────────────┼───────────────┼──────────────┤
│ Lando Norris      │     4 │            91.174 │             6 │  